# Семинар 6. Свёрточные слои и свёрточные сети

**Внимание!** Для работы над задачами этого семинара на локальном устройстве потребуется установить [`PyTorch`](https://pytorch.org/get-started/locally/), если он ещё не был установлен вами при работе над задачами Семинара 5, а также [`Pillow`](https://pillow.readthedocs.io/en/stable/) для работы с изображениями.

При работе в Google Colab ничего устанавливать не нужно, модули уже доступны и их можно сразу импортировать.

После установки `PyTorch` убедитесь, что следующий код запускается и выводит на экран набор нормально распределенных вещественных чисел размером 5×3:

```python
import torch
x = torch.rand(5, 3)
print(x)
```

## Задача 6.1. Свёрточные слои [max = 30 баллов]

### Формулировка задания

Реализуйте с помощью библиотеки `numpy`:
- [15 баллов] одномерную свёрточную функцию с сигнатурой, аналогичной [`conv1d`](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.conv1d.html) из `torch.nn.functional`
- [15 баллов] двумерную свёрточную функцию с сигнатурой, аналогичной [`conv2d`](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.conv2d.html) из `torch.nn.functional`

В обоих случаях функции должны принимать и возвращать массивы `numpy.ndarray` с типом данных `np.float32` корректного размера.

### Критерии оценки

При выполнении задания разрешается пользоваться описанием тестов (приведено ниже): ограничиться простой базовой реализацией, которая пойдет некоторое количество тестов подряд, не реализовывая весь функционал сразу.

Скачайте данные для тестирования и запустите ваши функции на них (см. шаблоны ниже).

При условии, что решение прошло очную защиту, по каждому из пунктов балл выставляется по формуле (максимальный балл) × (доля пройденных подряд тестов) с округлением до десятых. Реализация двумерной свёрточной функции оценивается только при условии успешной (≥ 40% пройденных подряд тестов) реализации одномерной свёрточной функции.

### Описание тестов

Сначала обязательно проверяется сигнатура функции. Порядок, наименование аргументов, обязательные и опциональные аргументы, значения по умолчанию опциональных аргументов должны обязательно совпадать с реализацией из `torch`, иначе тестирование прерывается.

Далее генерируется набор данных, на котором по очереди тестируется:
- корректность размерности и типа выхода для нескольких входов без учета совпадения значений со значением всех опциональных аргументов по умолчанию
- корректность значений выхода для нескольких входов со значением всех опциональных аргументов по умолчанию
- корректность значений выхода для нескольких входов при передаче нетривиальных значений опциональных аргументов `bias`, `stride` и `padding`, тогда как `dilation` и `groups` зафиксированы на значениях по умолчанию
- корректность значений выхода для нескольких входов при передаче нетривиальных значений опциональных аргументов `bias`, `stride`, `padding`, `dilation` и `groups`

При проваливании одного из тестов тестирование прерывается, фиксируется доля пройденных тестов.

### Ваше решение

In [ ]:
import numpy as np

def _to_int(v):
    """Преобразует аргумент (int, float, ndarray, list, tuple) в int."""
    if isinstance(v, (np.ndarray, list, tuple)):
        v = v[0]
    if hasattr(v, "item"):
        return int(v.item())
    return int(v)

# Реализация conv1d

def numpy_conv1d(
    input,
    weight,
    bias=None,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
):

    x = input.astype(np.float32)
    w = weight.astype(np.float32)

    # Обработка stride и dilation
    try:
        if isinstance(stride, (tuple, list)):
            stride = _to_int(stride[0])
        else:
            stride = _to_int(stride)

        if isinstance(dilation, (tuple, list)):
            dilation = _to_int(dilation[0])
        else:
            dilation = _to_int(dilation)
    except ValueError as e:
        # Может возникнуть, если _to_int получает нечисловой аргумент
        raise TypeError(f"Нечисловой аргумент для stride/dilation: {e}") from e

    N, C_in, L_in = x.shape
    C_out, C_per_group, K = w.shape

    # Проверки групп
    if C_in % groups != 0:
        raise ValueError("C_in must be divisible by groups")
    if C_out % groups != 0:
        raise ValueError("C_out must be divisible by groups")

    Cin_g = C_in // groups
    Cout_g = C_out // groups

    if C_per_group != Cin_g:
        raise ValueError("weight shape doesn't match input and groups")

    # Обработка Padding
    padding_left = 0
    padding_right = 0

    if isinstance(padding, str):
        mode = padding.lower()
        K_eff = dilation * (K - 1) + 1

        if mode == "valid":
            padding_left = 0
            padding_right = 0
        elif mode == "same":
            L_out_est = int(np.ceil(L_in / stride))
            pad_total = max(0, (L_out_est - 1) * stride + K_eff - L_in)
            padding_left = pad_total // 2
            padding_right = pad_total - padding_left
        else:
            raise ValueError(f"unknown padding string: {padding}")
    else:
        try:
            if isinstance(padding, (tuple, list)):
                padding_left = padding_right = _to_int(padding[0])
            else:
                padding_left = padding_right = _to_int(padding)
        except ValueError as e:
             raise TypeError(f"Нечисловой аргумент для padding: {e}") from e

    # Вычисление размера выхода
    L_out = (L_in + padding_left + padding_right - dilation * (K - 1) - 1) // stride + 1

    if L_out <= 0:
        return np.zeros((N, C_out, max(L_out, 0)), dtype=np.float32)

    # Паддинг и инициализация выхода
    x_pad = np.pad(
        x,
        ((0, 0), (0, 0), (padding_left, padding_right)),
        mode="constant"
    )

    out = np.zeros((N, C_out, L_out), dtype=np.float32)

    # Индексы
    ks = np.arange(K) * dilation

    # Выполнение свёртки с использованием матричного умножения
    for g in range(groups):
        in_g = x_pad[:, g * Cin_g:(g + 1) * Cin_g, :]
        w_g = w[g * Cout_g:(g + 1) * Cout_g, :, :]
        w_col = w_g.reshape(Cout_g, -1)

        for out_idx in range(L_out):
            start = out_idx * stride
            idx = start + ks
            patch = in_g[:, :, idx]
            patch_flat = patch.reshape(N, -1)

            # Матричное умножение
            conv_result = patch_flat @ w_col.T

            out[:, g * Cout_g:(g + 1) * Cout_g, out_idx] = conv_result

    # Добавляем смещение
    if bias is not None:
        out += np.asarray(bias, dtype=np.float32)[None, :, None]

    return out


In [ ]:
# Реализация conv2d
def numpy_conv2d(
    input,
    weight,
    bias=None,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
):
    x = input.astype(np.float32)
    w = weight.astype(np.float32)
    if bias is not None:
        bias = np.asarray(bias, dtype=np.float32)

    # Обработка аргументов
    def _to_2tuple(arg, name):
        if isinstance(arg, str):
            # Строковый паддинг 'same'/'valid' обрабатывается ниже
            if name == 'padding':
                return arg, arg
            raise TypeError(f"Строковый аргумент для {name} не поддерживается")

        if isinstance(arg, int):
            return _to_int(arg), _to_int(arg)
        if isinstance(arg, (tuple, list)):
            if len(arg) == 1:
                return _to_int(arg[0]), _to_int(arg[0])
            if len(arg) == 2:
                return _to_int(arg[0]), _to_int(arg[1])
        return _to_int(arg), _to_int(arg) # Попытка преобразовать неожиданный тип

    stride_h, stride_w = _to_2tuple(stride, 'stride')
    dilation_h, dilation_w = _to_2tuple(dilation, 'dilation')
    pad_arg_h, pad_arg_w = _to_2tuple(padding, 'padding') # Может быть строкой

    N, C_in, H_in, W_in = x.shape
    C_out, C_per_group, K_H, K_W = w.shape

    # Проверки групп
    if C_in % groups != 0 or C_out % groups != 0:
        raise ValueError("C_in and C_out must be divisible by groups")
    Cin_g = C_in // groups
    Cout_g = C_out // groups
    if C_per_group != Cin_g:
        raise ValueError("weight shape doesn't match input and groups")

    # Обработка Padding (с поддержкой строк 'same'/'valid')
    def calculate_padding_2d(L_in, K, stride, dilation, pad_arg):
        pad_left = 0
        pad_right = 0
        if isinstance(pad_arg, str):
            mode = pad_arg.lower()
            K_eff = dilation * (K - 1) + 1
            if mode == "valid":
                pad_left = pad_right = 0
            elif mode == "same":
                L_out_est = int(np.ceil(L_in / stride))
                pad_total = max(0, (L_out_est - 1) * stride + K_eff - L_in)
                pad_left = pad_total // 2
                pad_right = pad_total - pad_left
            else:
                raise ValueError(f"unknown padding string: {pad_arg}")
        else:
            # Числовой паддинг
            pad_val = _to_int(pad_arg)
            pad_left = pad_right = pad_val
        return pad_left, pad_right

    pad_h_top, pad_h_bottom = calculate_padding_2d(H_in, K_H, stride_h, dilation_h, pad_arg_h)
    pad_w_left, pad_w_right = calculate_padding_2d(W_in, K_W, stride_w, dilation_w, pad_arg_w)

    # Вычисление размера выхода
    H_out = (H_in + pad_h_top + pad_h_bottom - dilation_h * (K_H - 1) - 1) // stride_h + 1
    W_out = (W_in + pad_w_left + pad_w_right - dilation_w * (K_W - 1) - 1) // stride_w + 1

    if H_out <= 0 or W_out <= 0:
        return np.zeros((N, C_out, max(H_out, 0), max(W_out, 0)), dtype=np.float32)

    # Паддинг и инициализация выхода
    x_pad = np.pad(
        x,
        ((0, 0), (0, 0), (pad_h_top, pad_h_bottom), (pad_w_left, pad_w_right)),
        mode="constant"
    )

    out = np.zeros((N, C_out, H_out, W_out), dtype=np.float32)

    # Индексы
    kh_indices = np.arange(K_H) * dilation_h
    kw_indices = np.arange(K_W) * dilation_w

    # Размеры для матричного умножения
    patch_size = Cin_g * K_H * K_W

    # Выполнение свёртки с использованием матричного умножения
    for g in range(groups):
        in_g = x_pad[:, g * Cin_g:(g + 1) * Cin_g, :, :]
        w_g = w[g * Cout_g:(g + 1) * Cout_g, :, :, :]
        w_col = w_g.reshape(Cout_g, -1)

        for h_out in range(H_out):
            for w_out in range(W_out):
                h_start = h_out * stride_h
                w_start = w_out * stride_w
                h_idx = h_start + kh_indices
                w_idx = w_start + kw_indices

                patch = in_g[:, :, h_idx[:, np.newaxis], w_idx[np.newaxis, :]]
                patch_flat = patch.reshape(N, -1)

                # Матричное умножение
                conv_result = patch_flat @ w_col.T

                out[:, g * Cout_g:(g + 1) * Cout_g, h_out, w_out] = conv_result

    # Добавляем смещение
    if bias is not None:
        out += bias[None, :, None, None]

    return out

### Тестирование

#### Утилиты для тестирования
**Внимание!** Не изменяйте код в ячейке ниже! Только запустите её.

In [ ]:
from inspect import signature, _empty
from pickle import load
from torch import from_numpy
from torch.nn.functional import conv1d, conv2d
import warnings
warnings.formatwarning = lambda msg, *args, **kwargs: f"Warning: {str(msg)}"

def test_signature(conv)->bool:
    if not callable(conv):
        warnings.warn("исследуемый объект не является функцией")
        return False

    expected_parameters = {
        "input": {"optional": False},
        "weight": {"optional": False},
        "bias": {"optional": True, "default": None},
        "stride": {"optional": True, "default": 1},
        "padding": {"optional": True, "default": 0},
        "dilation": {"optional": True, "default": 1},
        "groups": {"optional": True, "default": 1},
    }

    implemented_parameters = list(signature(conv).parameters.values())

    if len(implemented_parameters) != len(expected_parameters):
        warnings.warn("исследуемая функция содержит некорректное число параметров")
        return False

    for (name, info), param in zip(expected_parameters.items(), implemented_parameters):

        if param.name != name:
            warnings.warn(f"аргумент {name} отсутствует или находится на некорректной позиции")
            return False

        elif (info["optional"]) and (param.default != info["default"]):
            warnings.warn(f"значение по умолчанию для аргумента {name} некорректно")
            return False

        elif (not info["optional"]) and (param.default != _empty):
            warnings.warn(f"для обязательного аргумента {name} указано значение по умолчанию")
            return False

        else:
            continue

    return True

def preprocess_kwargs(kwargs, mode="numpy")->dict:
    output = dict()
    for key, val in kwargs.items():
        if isinstance(val, np.ndarray):
            if mode == "numpy":
                output[key] = val.astype(np.float32)
            elif mode == "torch":
                output[key] = from_numpy(val)
            else:
                raise ValueError(f"неизвестный режим предобработки mode={mode}")
        elif isinstance(val, (int, tuple, str)):
            output[key] = val
        elif val is None:
            output[key] = None
        else:
            raise TypeError(f"некорректный тип данных: type({key})={type(val)}")
    return output


def test_runtime(numpy_conv, torch_conv, list_of_kwargs)->tuple[int, int]:

    tot = len(list_of_kwargs)
    ok = True

    for i, kwargs in enumerate(list_of_kwargs):

        try:
            numpy_output = numpy_conv(**preprocess_kwargs(kwargs, mode="numpy"))

        except Exception as e:
            text = f"{(i+1)} тест   из {tot}: обнаружено исключение "
            if hasattr(e, "msg"):
                text += f"({e.msg})"
            else:
                text += f"({str(e)})"
            warnings.warn(text)
            ok = False
            break

        torch_output = torch_conv(**preprocess_kwargs(kwargs, mode="torch")).numpy().astype(np.float32)

        if not isinstance(numpy_output, np.ndarray):
            warnings.warn(f"{(i+1)} тест   из {tot}: обнаружен некорректный тип возвращаемого значения")
            ok = False
            break

        elif numpy_output.dtype != np.float32:
            warnings.warn(f"{(i+1)} тест   из {tot}: обнаружен некорректный тип элементов массива")
            ok = False
            break

        elif numpy_output.shape != torch_output.shape:
            warnings.warn(f"{(i+1)} тест   из {tot}: обнаружен массив некорректной формы")
            ok = False
            break

        elif (i >= (tot // 10)) and (not np.allclose(numpy_output, torch_output, rtol=1.0e-3, atol=1.0e-3)):
            warnings.warn(f"{(i+1)} тест   из {tot}: обнаружены некорректные значения массива")
            ok = False
            break

    if ok:
        return (tot, tot)
    else:
        return (i, tot)


def test_pipeline(numpy_conv, torch_conv, file_path, max_score)->float:

    with open(file_path, "rb") as f:
        list_of_kwargs = load(f)

    try:
        assert test_signature(numpy_conv), "базовый тест сигнатуры не пройден"
        success, total = test_runtime(numpy_conv, torch_conv, list_of_kwargs)
        print(f"пройдено {success} тестов из {total}"),
        score = round(max_score * success / total, 1)

    except Exception as e:
        print(e)
        score = 0.0

    return score

#### Загрузка тестовых данных


Перед тестированием скачайте архив c тестовыми данными по [ссылке](https://drive.google.com/file/d/1axUyoRxDdSrXEzqRpuicGD8CrKJviEta/view?usp=sharing), распакуйте и переместите файлы в рабочую директорию вашего блокнота/скрипта.



> <font color="#DC143C">**ВНИМАНИЕ!**</font> Тестовые данные обновлены 27.11.2025 в 22:05 (доступны по той же ссылке). Предыдущая версия тестовых данных была некорректа. Перезапустите тесты, если вы пользовались предыдущей версией данных

> <font color="#DC143C">**ВНИМАНИЕ!**</font> Тестовая утилита обновлена 28.11.2025 в 12:20. Повышен допустимый уровень ошибки с $10^{-5}$ до $10^{-3}$. Перезапустите тесты



Для перестраховки вы можете для перестраховки проверить размер загруженных файлов. Например, если на вашем устройстве доступна утилита `du` (в Google Colab доступна по умолчанию):



```shell
$ du -h test_conv1d.pickle
1.2M    test_conv1d.pickle

$ du -h test_conv2d.pickle
42M     test_conv2d.pickle
```
При значительном расхождении размера файлов в любую сторону обратитесь к преподавателю.


#### Тест `conv1d`



In [ ]:
score_conv1d = test_pipeline(numpy_conv1d, conv1d, "test_conv1d.pickle", 15)

пройдено 100 тестов из 100


In [ ]:
print(f"балл за реализацию conv1d: {score_conv1d:.1f}")

балл за реализацию conv1d: 15.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Тест `conv2d`

In [ ]:
score_conv2d = test_pipeline(numpy_conv2d, conv2d, "test_conv2d.pickle", 15)
score_conv2d = score_conv2d if (score_conv1d >= 6.0) else 0.0

пройдено 100 тестов из 100


In [ ]:
print(f"балл за реализацию conv2d: {score_conv2d:.1f}")

балл за реализацию conv2d: 15.0


## Задача 6.2 [max = 15 баллов]

**Внимание!** Данная задача оценивается только при условии успешного выполнения задачи 6.1 (≥40% пройденных подряд тестов для одномерной и двумерной свёрточных функций).



[5 баллов] Выберите 2–3 цветные фотографии из разных разделов [галереи](https://commons.wikimedia.org/wiki/Commons:Featured_pictures/Natural_phenomena) природных явлений на Wikimedia Commons.

С помощью модуля [`PIL.Image`](https://pillow.readthedocs.io/en/stable/reference/Image.html) загрузите и обесцветьте этим фотографии. Далее с помощью созданной вами в предыдущей задаче реализации сверточного фильтра примените к этим фотографиями горизонтальный и вертикальный [операторы Собеля](https://en.wikipedia.org/wiki/Sobel_operator). Для каждой из фотографий визуализируйте рядом на одном изображении:
- исходное цветное фото
- обесцвеченное фото
- результат применения горизонтального оператора Собеля к обесцвеченному фото
- результат применения вертикального оператора Собеля к обесцвеченному фото

[5 баллов] Результат применения горизонтального и вертикального фильтра Собеля можно воспринимать как дискретизованную оценку полей компонент градиента поля яркости исходного изображения, $G_x(x_i, y_j)$ и $G_y(x_i, y_j)$. Можно перейти в полярные координаты:
$$
G_x = |G| \cos (\arg G), \quad G_y = |G| \sin (\arg G),
$$

и таким образом ввести абсолютную величину градиента $|G|$ и угол, определяющий его направление, $\arg G \in [0, 2\pi)$. Визуализируйте карты $|G|$ и $\arg G$ для каждого фото по аналогии с предыдущим пунктом. Для $\arg G$ используйте циклическую цветовую карту, чтобы показать периодичность.

[5 баллов] Для тех же фотографий дополнительно изобразите векторное поле $G=(G_x, G_y)$ при помощи [`matplotlib.pyplot.quiver`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html). Отрегулируйте длину стрелок для удобства визуализации.

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from scipy.ndimage import convolve

# Ядра Собеля
SOBEL_X = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)

SOBEL_Y = np.array([
    [1, 2, 1],
    [0, 0, 0],
    [-1, -2, -1]
], dtype=np.float32)

def show_quiver(Gx, Gy, background_img, step=4, scale=20, title="Gradient field"):
    H, W = Gx.shape
    # Создаем сетку координат
    y, x = np.mgrid[0:H:step, 0:W:step]

    # Прореженные, не нормированные компоненты градиента
    u = Gx[::step, ::step]
    v = Gy[::step, ::step]

    plt.figure(figsize=(8, 8))
    plt.imshow(background_img, cmap="gray")

    # Векторное поле
    plt.quiver(x, y, u, v,
               angles="xy",
               scale_units="xy",
               scale=scale,
               color="red",
               pivot='middle',
               headwidth=5)

    plt.gca().set_aspect("equal")
    plt.axis("off")
    plt.title(title)
    plt.tight_layout()
    plt.show()

def process_image(file_path):
    print(f"--- Обработка файла: {file_path} ---")

    # Загрузить картинку из диска
    try:
        color_img = Image.open(file_path).convert("RGB")
        max_size = 500
        if color_img.width > max_size or color_img.height > max_size:
            color_img.thumbnail((max_size, max_size))

        grayscale_img = color_img.convert("L")
    except FileNotFoundError:
        print(f"Ошибка: Файл не найден по пути {file_path}")
        return
    except Exception as e:
        print(f"Ошибка при загрузке/конвертации изображения: {e}")
        return

    # Преобразование цветного и обесцвеченного изображений в массивы NumPy
    color_array = np.array(color_img, dtype=np.float32)
    gray_array = np.array(grayscale_img, dtype=np.float32)

    # Рассчёт градиента
    H, W, _ = color_array.shape
    Gx_multi = np.zeros((H, W), dtype=np.float32)
    Gy_multi = np.zeros((H, W), dtype=np.float32)

    for i in range(3):
        channel = color_array[:, :, i]
        Gx_channel = convolve(channel, SOBEL_X, mode='nearest')
        Gy_channel = convolve(channel, SOBEL_Y, mode='nearest')
        G_mag_channel = np.sqrt(Gx_channel**2 + Gy_channel**2)
        current_mag_sq = Gx_multi**2 + Gy_multi**2
        new_mag_sq = G_mag_channel**2

        mask = new_mag_sq > current_mag_sq
        Gx_multi = np.where(mask, Gx_channel, Gx_multi)
        Gy_multi = np.where(mask, Gy_channel, Gy_multi)

    Gx = Gx_multi
    Gy = Gy_multi

    # Вычисляем величины градиента и угла градиента
    G_magnitude = np.sqrt(Gx**2 + Gy**2)
    G_mag_normalized = G_magnitude / G_magnitude.max() if G_magnitude.max() != 0 else G_magnitude
    G_angle = np.arctan2(Gy, Gx)
    G_angle_positive = (G_angle + np.pi) % (2 * np.pi)

    # Визуализация результатов
    Gx_vis = np.abs(Gx); Gx_vis = Gx_vis / Gx_vis.max() if Gx_vis.max() != 0 else Gx_vis
    Gy_vis = np.abs(Gy); Gy_vis = Gy_vis / Gy_vis.max() if Gy_vis.max() != 0 else Gy_vis
    plt.figure(figsize=(16, 4)); plt.suptitle(f"Часть 1: Результаты операторов Собеля для {file_path}", fontsize=16)
    plt.subplot(1, 4, 1); plt.imshow(color_img); plt.title("1. Исходное цветное фото"); plt.axis('off')
    plt.subplot(1, 4, 2); plt.imshow(grayscale_img, cmap='gray'); plt.title("2. Обесцвеченное фото"); plt.axis('off')
    plt.subplot(1, 4, 3); plt.imshow(Gx_vis, cmap='gray'); plt.title("3. Результат Gx"); plt.axis('off')
    plt.subplot(1, 4, 4); plt.imshow(Gy_vis, cmap='gray'); plt.title("4. Результат Gy"); plt.axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()

    H_angle = G_angle_positive / (2 * np.pi); HSV_img = np.stack([H_angle, np.ones_like(H_angle), np.ones_like(H_angle)], axis=-1)
    argG_color_map = hsv_to_rgb(HSV_img)
    plt.figure(figsize=(12, 6)); plt.suptitle(f"Часть 2: Величина и Угол Градиента для {file_path}", fontsize=16)
    ax1 = plt.subplot(1, 2, 1); mag_im = ax1.imshow(G_mag_normalized, cmap='viridis'); ax1.set_title("$|G|$"); plt.colorbar(mag_im, ax=ax1, label='Нормализованная величина'); ax1.axis('off')
    ax2 = plt.subplot(1, 2, 2); arg_im = ax2.imshow(argG_color_map); ax2.set_title("$\\arg G$ - Циклическая карта")
    cbar_labels = [f'{i}°' for i in range(0, 360, 45)]; cmap_hsv = plt.colormaps['hsv']; sm = plt.cm.ScalarMappable(cmap=cmap_hsv); sm.set_array(G_angle_positive)
    cbar = plt.colorbar(sm, ax=ax2, ticks=np.linspace(0, 2*np.pi, len(cbar_labels)), orientation='vertical')
    cbar.ax.set_yticklabels(cbar_labels); cbar.set_label('Угол градиента в плоскости [0° - 360°]'); ax2.axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.show()
    show_quiver(Gx, Gy, gray_array, step=4, scale=80, title=f"Часть 3: Векторное поле Градиента для {file_path}")

image_files = ["image1.jpg", "image2.jpg", "image3.jpg"]

for file in image_files:
    process_image(file)